## 04 - Avaliação e Comparação dos Modelos

Avaliar performance dos modelos no conjunto de teste

# Imports

In [ ]:
import sys
sys.path.append('/home/jovyan/work')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src import S3Client, ModelEvaluator, MLFlowClient, DatabaseClient

# Carregar dados de teste

In [ ]:
s3 = S3Client()

X_test = s3.read_csv('processed/X_test_scaled.csv')
y_test = s3.read_csv('processed/y_test.csv')['target']

print(f"📊 X_test: {X_test.shape}")
print(f"📊 y_test: {y_test.shape}")

# Inicializar evaluator e MLFlow

In [ ]:
evaluator = ModelEvaluator()
mlflow_client = MLFlowClient(experiment_name="heart-disease-prediction")

# Carregar Modelos Treinados do MLFlow

In [ ]:
print("📋 Modelos registrados no MLFlow:")

# Modelos do artigo
article_models = ['knn', 'random_forest', 'logistic_regression', 
                  'svm', 'naive_bayes', 'decision_tree']

# Modelos de melhoria
improved_models = ['gradient_boosting', 'random_forest_tuned']

all_model_names = article_models + improved_models

for name in all_model_names:
    print(f"   - {name}")

# Avaliar Modelos do Artigo

In [ ]:
results = []

for model_name in all_model_names:
    try:
        print(f"\n🔄 Avaliando: {model_name}")
        
        # Carregar modelo do MLFlow
        model_uri = f"models:/{model_name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        if model is None:
            print(f"   ⚠️ Modelo não encontrado: {model_name}")
            continue
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
        
        # Avaliar
        metrics = evaluator.evaluate_model(
            y_test, 
            y_pred, 
            y_pred_proba,
            model_name=model_name
        )
        
        results.append(metrics)
        
        # Salvar métricas no banco
        db = DatabaseClient()
        db.insert_model_metrics(
            model_name=model_name,
            model_version="1",
            metrics=metrics
        )
        
    except Exception as e:
        print(f"   ❌ Erro ao avaliar {model_name}: {str(e)}")

print(f"\n✅ {len(results)} modelos avaliados!")

# Comparação de Modelos

In [ ]:
comparison_df = evaluator.compare_models(results)

print("\n📊 Comparação de Modelos:")
display(comparison_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']])

In [ ]:
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1_score']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    # Ordenar por métrica
    df_sorted = comparison_df.sort_values(metric, ascending=False)
    
    # Cores diferentes para modelos do artigo vs melhorias
    colors = ['#3498db' if name in article_models else '#e74c3c' 
              for name in df_sorted['model_name']]
    
    ax.barh(df_sorted['model_name'], df_sorted[metric], color=colors)
    ax.set_xlabel(metric.capitalize())
    ax.set_title(f'Comparação - {metric.upper()}')
    ax.invert_yaxis()
    
    # Adicionar valores
    for i, v in enumerate(df_sorted[metric]):
        ax.text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()


# Melhor Modelo

In [ ]:
best_metrics = evaluator.get_best_model_metrics(metric='accuracy')

print(f"\n🏆 Melhor Modelo: {best_metrics['model_name']}")
print(f"   Accuracy:  {best_metrics['accuracy']:.4f}")
print(f"   Precision: {best_metrics['precision']:.4f}")
print(f"   Recall:    {best_metrics['recall']:.4f}")
print(f"   F1-Score:  {best_metrics['f1_score']:.4f}")
if best_metrics.get('roc_auc'):
    print(f"   ROC AUC:   {best_metrics['roc_auc']:.4f}")

## Matrizes de Confusão

In [ ]:
top_4_models = comparison_df.head(4)['model_name'].tolist()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, model_name in enumerate(top_4_models):
    try:
        # Carregar modelo
        model_uri = f"models:/{model_name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        # Predições
        y_pred = model.predict(X_test)
        
        # Matriz de confusão
        cm = evaluator.get_confusion_matrix(y_test, y_pred)
        
        # Plotar
        ax = axes[idx]
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                   xticklabels=['Saudável', 'Doença'],
                   yticklabels=['Saudável', 'Doença'])
        ax.set_title(f'Matriz de Confusão - {model_name}')
        ax.set_ylabel('Real')
        ax.set_xlabel('Predito')
        
    except Exception as e:
        print(f"❌ Erro ao plotar CM para {model_name}: {str(e)}")

plt.tight_layout()
plt.show()

## Curvas ROC

In [ ]:
plt.figure(figsize=(12, 8))

for model_name in top_4_models:
    try:
        # Carregar modelo
        model_uri = f"models:/{model_name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        # Probabilidades
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)
            
            # Calcular ROC
            roc_data = evaluator.calculate_roc_curve(y_test, y_pred_proba)
            
            if roc_data:
                plt.plot(roc_data['fpr'], roc_data['tpr'], 
                        label=f"{model_name} (AUC={roc_data['auc']:.3f})",
                        linewidth=2)
        
    except Exception as e:
        print(f"❌ Erro ao plotar ROC para {model_name}: {str(e)}")

plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.500)', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curvas ROC - Comparação dos Melhores Modelos')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()

# Classification Reports

print("\n" + "="*60)
print("📊 CLASSIFICATION REPORTS - TOP 3 MODELOS")
print("="*60)

for model_name in comparison_df.head(3)['model_name']:
    try:
        # Carregar modelo
        model_uri = f"models:/{model_name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        # Predições
        y_pred = model.predict(X_test)
        
        # Report
        print(f"\n{'='*60}")
        print(f"MODELO: {model_name.upper()}")
        print(f"{'='*60}")
        
        report = evaluator.get_classification_report(
            y_test, 
            y_pred,
            target_names=['Saudável (0)', 'Doença (1)']
        )
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")

### Feature Importance (Top 3 Tree-based Models)

In [ ]:
tree_models = ['random_forest', 'decision_tree', 'gradient_boosting', 'random_forest_tuned']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

feature_names = X_test.columns.tolist()

for idx, model_name in enumerate(tree_models):
    try:
        # Carregar modelo
        model_uri = f"models:/{model_name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        if hasattr(model, 'feature_importances_'):
            # Feature importance
            importances = model.feature_importances_
            
            # Criar DataFrame
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False).head(15)
            
            # Plotar
            ax = axes[idx]
            ax.barh(importance_df['feature'], importance_df['importance'], color='steelblue')
            ax.set_xlabel('Importância')
            ax.set_title(f'Feature Importance - {model_name}')
            ax.invert_yaxis()
            
    except Exception as e:
        print(f"❌ Erro ao plotar FI para {model_name}: {str(e)}")

plt.tight_layout()
plt.show()

# Exportar Resultados

In [ ]:
evaluator.export_metrics('/home/jovyan/work/models/model_metrics.csv')

# Upload para MinIO
s3.upload_file('/home/jovyan/work/models/model_metrics.csv', 'models/model_metrics.csv')

print("✅ Métricas exportadas para MinIO!")

# Resumo Final

In [ ]:
# Resumo estatístico
summary = evaluator.get_metrics_summary()

print("\n📊 Resumo Estatístico das Métricas:")
display(summary[['accuracy', 'precision', 'recall', 'f1_score']])

# Comparação Artigo vs Melhorias
print("\n📊 Comparação: Modelos do Artigo vs Melhorias")

article_results = comparison_df[comparison_df['model_name'].isin(article_models)]
improved_results = comparison_df[comparison_df['model_name'].isin(improved_models)]

print("\n🎯 MODELOS DO ARTIGO:")
print(f"   Accuracy média: {article_results['accuracy'].mean():.4f}")
print(f"   Melhor: {article_results.iloc[0]['model_name']} ({article_results.iloc[0]['accuracy']:.4f})")

print("\n🚀 MODELOS DE MELHORIA:")
print(f"   Accuracy média: {improved_results['accuracy'].mean():.4f}")
print(f"   Melhor: {improved_results.iloc[0]['model_name']} ({improved_results.iloc[0]['accuracy']:.4f})")